In [4]:
import socket
import ssl
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives.serialization import load_pem_private_key, load_pem_public_key
from cryptography.hazmat.primitives.asymmetric import rsa
from cryptography.hazmat.primitives.asymmetric import padding
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.primitives.serialization import load_pem_private_key
from cryptography.hazmat.primitives.asymmetric import dh
from cryptography.hazmat.primitives import serialization
import os
import tkinter as tk



In [5]:
encryption_choice = 0
msg = ""

def generate_rsa_keypair():
    private_key = rsa.generate_private_key(
        public_exponent=65537,
        key_size=2048,
        backend=default_backend()
    )
    return private_key

def generate_dh_keypair():
    parameters = dh.generate_parameters(generator=2, key_size=2048)
    private_key = parameters.generate_private_key()
    public_key = private_key.public_key()
    return private_key, public_key

# def derive_dh_shared_key(private_key, public_key):
#     shared_key = private_key.exchange(public_key)
#     return shared_key

def derive_dh_shared_key(private_key, public_key):
    shared_key = public_key.public_numbers().y ** private_key.private_numbers().x % public_key.public_numbers().parameter_numbers.g
    return shared_key.to_bytes((shared_key.bit_length() + 7) // 8, byteorder='big')


def rsa_encrypt(public_key, plaintext):
    ciphertext = public_key.encrypt(
        plaintext.encode(),
        padding.OAEP(
        mgf=padding.MGF1(algorithm=hashes.SHA256()),
        algorithm=hashes.SHA256(),
        label=None)
        )
    return ciphertext

def rsa_decrypt(private_key, ciphertext):
    plaintext = private_key.decrypt(
    ciphertext,
    padding.OAEP( mgf=padding.MGF1(algorithm=hashes.SHA256()),
    algorithm=hashes.SHA256(),
    label=None))
    return plaintext.decode()

def aes_encrypt(key, plaintext):
    iv = os.urandom(16)
    cipher = Cipher(algorithms.AES(key), modes.CFB(iv), backend=default_backend())
    encryptor = cipher.encryptor()
    ciphertext = encryptor.update(plaintext.encode()) + encryptor.finalize()
    return iv + ciphertext

def aes_decrypt(key, ciphertext):
    iv = ciphertext[:16]
    ciphertext = ciphertext[16:]
    cipher = Cipher(algorithms.AES(key), modes.CFB(iv), backend=default_backend())
    decryptor = cipher.decryptor()
    plaintext = decryptor.update(ciphertext) + decryptor.finalize()
    return plaintext.decode()

def start_client():
    client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    context = ssl.SSLContext(ssl.PROTOCOL_TLS_CLIENT)
    context.load_verify_locations(cafile="server.crt")
    context.check_hostname = False
    client_socket.connect(("localhost", 8443))
    print("Connected to server.")
    
    with context.wrap_socket(client_socket, server_hostname="localhost") as ssock:
        
        print("SSL/TLS connection established.")
        
        encryption_choice = input("Enter encryption method (1 for RSA, 2 for AES): ")
        ssock.send(encryption_choice.encode())
      
    
        if encryption_choice == "1":
            # RSA encryption and PKI key exchange
            private_key = generate_rsa_keypair()
            public_key = private_key.public_key()
            print(f"Public key generated. {public_key}")
            ssock.send(public_key.public_bytes(
                encoding=serialization.Encoding.PEM,
                format=serialization.PublicFormat.SubjectPublicKeyInfo
            ))
            
            #receive public key from server
            received_public_key = ssock.recv(2048)
            server_public_key = load_pem_public_key(received_public_key, backend=default_backend())
            
            while True:
                #send msg to server
                msg_for_server = input("Enter your message for server: ")
            
                #encrypt msg with server's public key
                msg_for_server = rsa_encrypt(server_public_key, msg_for_server)
                print(f"Encrypted RSA message for server: {msg_for_server}")
                #send encrypted msg to server
                ssock.send(msg_for_server)
                print("Message sent to server.")
                
                #-----------------------------------------------
                #receive msg from server
                msg_from_server = ssock.recv(2048)
                msg_from_server = rsa_decrypt(private_key, msg_from_server)
                print(f"Decrypted RSA message from server: {msg_from_server}")
                if msg_from_server == "exit":
                    break

        elif encryption_choice == "2":
            print("Diffie-Hellman key exchange initiated.")
            # AES encryption and Diffie-Hellman key exchange
            private_key, public_key = generate_dh_keypair()
            print(f"Public key generated. {public_key}")
            
            ssock.send(public_key.public_bytes(
                encoding=serialization.Encoding.PEM,
                format=serialization.PublicFormat.SubjectPublicKeyInfo
            ))
            print("Public key sent to server.")

            received_public_key = ssock.recv(2048)
            print(f"Received public key from server: {received_public_key}")
            received_public_key = serialization.load_pem_public_key(received_public_key, backend=default_backend())
            
            shared_key = derive_dh_shared_key(private_key, received_public_key)
            print(f"Shared key: {shared_key}")

            while True: 
                msg_for_server = input("Enter your message for server: ")
                msg_for_server = aes_encrypt(shared_key, msg_for_server)
                ssock.send(msg_for_server)
                print("Message sent to server.")
                
                msg_from_server = ssock.recv(2048)
                msg_from_server = aes_decrypt(shared_key, msg_from_server)
                print(f"Decrypted AES message from server: {msg_from_server}")
                if msg_from_server == "exit":
                    break



In [6]:
if __name__ == "__main__":
   start_client()
   
#openssl req -x509 -newkey rsa:4096 -keyout ca.key -out ca.crt -days 365
   """
   info for ca.crt
   PEM pass phrase: 03455
   country name: PK
   state or province name: Punjab
   locality name: Rawalpindi
   organization name: FAST
   organizational unit name: Computing
   common name: FARQ
   email address: i200621@nu.edu.pk
   """



Connected to server.
SSL/TLS connection established.
Public key generated. <cryptography.hazmat.backends.openssl.rsa._RSAPublicKey object at 0x0000021A2E9D58D0>
Encrypted RSA message for server: b"\xb1[8\x8e.T\xc3\xb0\xe7\xa5\x92q<\x00%d\x05$\x80'\xe7_\xd2\xd9{\xe4i\xa2\xe7\xd2\x03\x88\xc4\xc6%\xf0\xe3\x1f\x06\xdfC\x17L\xb4\xe6\xc5\x1eB\x88\x07\xa1\xe5\xf4\xaa\x0b%GD\x10\x9c\xdf\xf4\xad9 \xa6@_\xa2}?\x1c\xb8U\xb0O\xc6\x0e\x96\xe2\x93i\xf54A1\x9d\xf8Lc\xd9\x11jc\xb8\x0f\xear\xcc\xc7\xa6\xff\x1b\x8d\xb9\xd7\xd8\xfa\x95\x9a\xa3\x1b+X\xd8\xfd\xcd\xd0\xce\xbeV\xc3\xcf\x9b\xecw\x02\xde*N\xa9H\xcd\xac\xc5FV.\x9b\x12+\xaa{j6\x8e\xc5<\xdd?\xbd\xb2\x0c\xd7\x84u\x9f\x7f\xb4a\x98\x82\xbf\xac/\x82q\x85\xab)pJY\xb3^V\x7f\x97\x02\xcc\xb7\x16`f^\xa1\xbf\xcc\x03\xc50\x11#\xd0\xef\xd9\x8e\x0e\xab,)\x9dw-\x00Q\x82O\xf6\xb9\xac,}j\x9a\xe4).\x8b\xe8\x13x{\xfb\xb3&F\xf2\x0bJAz\xde\x13dT\x19\x0f\xb9\xb8N<:\xd5\x05\xd5\x86\x1c\no\xd9\xe7\xf5\x92\xda\xcd"
Message sent to server.
Decrypted RSA message from serv

ValueError: Ciphertext length must be equal to key size.